In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [5]:
# 1. استدعاء المكتبات
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier

# 2. تحميل البيانات
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")
test_df = pd.read_csv("/kaggle/input/titanic/test.csv")

# الاحتفاظ بـ PassengerId من بيانات الاختبار
test_passenger_ids = test_df["PassengerId"]

# 3. دمج البيانات مؤقتًا للتنظيف الموحد
full_df = pd.concat([train_df, test_df], sort=False)

# 4. معالجة القيم المفقودة
full_df["Age"].fillna(full_df["Age"].median(), inplace=True)
full_df["Fare"].fillna(full_df["Fare"].median(), inplace=True)
full_df["Embarked"].fillna(full_df["Embarked"].mode()[0], inplace=True)
full_df["Cabin"].fillna("Unknown", inplace=True)

# 5. تحويل الأعمدة النصية إلى رقمية
label_cols = ["Sex", "Embarked", "Cabin"]
for col in label_cols:
    le = LabelEncoder()
    full_df[col] = le.fit_transform(full_df[col])

# 6. إنشاء ميزات إضافية
full_df["FamilySize"] = full_df["SibSp"] + full_df["Parch"] + 1
full_df["IsAlone"] = (full_df["FamilySize"] == 1).astype(int)
full_df["Age*Class"] = full_df["Age"] * full_df["Pclass"]
full_df["FarePerPerson"] = full_df["Fare"] / full_df["FamilySize"]

# 7. إسقاط الأعمدة غير المفيدة
drop_cols = ["PassengerId", "Name", "Ticket"]
full_df.drop(columns=drop_cols, inplace=True)

# 8. فصل البيانات مرة أخرى
train_clean = full_df.iloc[:len(train_df), :]
test_clean = full_df.iloc[len(train_df):, :]

X = train_clean.drop("Survived", axis=1)
y = train_clean["Survived"]

# 9. تقسيم البيانات لمجموعة تدريب واختبار
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 10. تعريف موديل CatBoost مع أفضل المعلمات
cat_model = CatBoostClassifier(
    iterations=200,
    depth=4,
    learning_rate=0.05,
    l2_leaf_reg=3,
    eval_metric='Accuracy',
    random_seed=42,
    verbose=100,
    early_stopping_rounds=50
)

# 11. تدريب الموديل
cat_model.fit(X_train, y_train, eval_set=(X_val, y_val))

# 12. تقييم الأداء على مجموعة التحقق
y_val_pred = cat_model.predict(X_val)
from sklearn.metrics import accuracy_score, classification_report
print("Accuracy on validation set:", accuracy_score(y_val, y_val_pred))
print(classification_report(y_val, y_val_pred))

# 13. التنبؤ على بيانات الاختبار
X_test_final = test_clean.drop("Survived", axis=1, errors='ignore')
y_test_pred = cat_model.predict(X_test_final)

# 14. إنشاء ملف submission
submission = pd.DataFrame({
    "PassengerId": test_passenger_ids,
    "Survived": y_test_pred.astype(int)
})

submission.to_csv("titanic_submission_4.csv", index=False)
print("🎉 ملف التسليم جاهز: titanic_submission.csv")


0:	learn: 0.7949438	test: 0.7932961	best: 0.7932961 (0)	total: 1.63ms	remaining: 324ms
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.8212290503
bestIteration = 26

Shrink model to first 27 iterations.
Accuracy on validation set: 0.8212290502793296
              precision    recall  f1-score   support

         0.0       0.82      0.91      0.86       110
         1.0       0.82      0.68      0.75        69

    accuracy                           0.82       179
   macro avg       0.82      0.80      0.80       179
weighted avg       0.82      0.82      0.82       179

🎉 ملف التسليم جاهز: titanic_submission.csv


/tmp/ipykernel_36/874793422.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  full_df["Age"].fillna(full_df["Age"].median(), inplace=True)
/tmp/ipykernel_36/874793422.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True